In [3]:
import os, shutil
import zipfile
import keras
from google.colab import files
files.upload()
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle (2).json


In [5]:
!kaggle competitions download -c digit-recognizer

 85% 62.0M/73.2M [00:00<00:00, 96.2MB/s]
100% 73.2M/73.2M [00:00<00:00, 165MB/s] 
 84% 41.0M/48.8M [00:00<00:00, 49.8MB/s]
100% 48.8M/48.8M [00:00<00:00, 70.7MB/s]
  0% 0.00/235k [00:00<?, ?B/s]
100% 235k/235k [00:00<00:00, 119MB/s]


In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras import layers
from keras import models
from keras.preprocessing import image
%matplotlib inline

In [0]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [0]:
train_data, validation_data = train_test_split(train_data, test_size = 0.1, shuffle=True)

x_train = (train_data.iloc[:,1:].values).astype('float32').reshape((-1, 28, 28, 1))
y_train = np.array(pd.get_dummies(train_data.iloc[:,0])).astype('float32')
x_val = validation_data.iloc[:, 1:].values.astype('float32').reshape((-1, 28, 28, 1))
y_val = np.array(pd.get_dummies(validation_data.iloc[:,0])).astype('float32')
x_test = test_data.iloc[:, :].values.astype('float32').reshape(-1, 28, 28, 1)

In [0]:
x_train /= 255
x_val /= 255
x_test /= 255

In [0]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))


model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))


model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dropout(0.5))

model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
datagen = image.ImageDataGenerator(
      featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)

In [0]:
datagen.fit(x_train)

In [0]:
batch_size = 100
epochs = 50

In [68]:
history = model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_val, y_val),
                              verbose = 2, steps_per_epoch=x_train.shape[0] // batch_size)

Epoch 1/50
 - 16s - loss: 0.3993 - acc: 0.8743 - val_loss: 0.1000 - val_acc: 0.9679
Epoch 2/50
 - 16s - loss: 0.2099 - acc: 0.9344 - val_loss: 0.0757 - val_acc: 0.9783
Epoch 3/50
 - 16s - loss: 0.1687 - acc: 0.9478 - val_loss: 0.0532 - val_acc: 0.9826
Epoch 4/50
 - 15s - loss: 0.1398 - acc: 0.9562 - val_loss: 0.0457 - val_acc: 0.9848
Epoch 5/50
 - 16s - loss: 0.1165 - acc: 0.9640 - val_loss: 0.0421 - val_acc: 0.9874
Epoch 6/50
 - 16s - loss: 0.1076 - acc: 0.9674 - val_loss: 0.0410 - val_acc: 0.9864
Epoch 7/50
 - 16s - loss: 0.0934 - acc: 0.9707 - val_loss: 0.0488 - val_acc: 0.9864
Epoch 8/50
 - 15s - loss: 0.0915 - acc: 0.9718 - val_loss: 0.0445 - val_acc: 0.9874
Epoch 9/50
 - 16s - loss: 0.0840 - acc: 0.9742 - val_loss: 0.0393 - val_acc: 0.9876
Epoch 10/50
 - 15s - loss: 0.0786 - acc: 0.9752 - val_loss: 0.0397 - val_acc: 0.9890
Epoch 11/50
 - 15s - loss: 0.0737 - acc: 0.9782 - val_loss: 0.0386 - val_acc: 0.9879
Epoch 12/50
 - 15s - loss: 0.0703 - acc: 0.9780 - val_loss: 0.0297 - val_a

In [69]:
model.evaluate(x_val, y_val)

4200/4200 [==============================] - 1s 146us/step


[0.03021612832955422, 0.9926190476190476]

In [0]:
predications = np.argmax(model.predict(x_test), 1)
result = pd.DataFrame({'ImageId': np.arange(len(predications)) + 1, 'Label': predications})
result.to_csv('submission.csv', index=False)

In [71]:
!kaggle competitions submit -c digit-recognizer -f submission.csv -m "Message"

Successfully submitted to Digit Recognizer

In [57]:
history = history.historyu
acc = history['acc']
loss = history['loss']
val_acc = history['val_acc']
val_loss = history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='train_acc')
plt.plot(epochs, val_acc, 'b', label='val_acc')
plt.title("Train_acc vs val_acc")
plt.xlabel('epochs')
plt.ylabel('acc')
plt.legend()

AttributeError: ignored